In [2]:
import urllib.request
from urllib.error import HTTPError
import requests
i = 0

links = []
while True:
    page = requests.get("https://bn.wikisource.org/w/index.php?title=Special:IndexPages&limit=5000&offset=" + str(i) + \
                        "&key=&order=")
    print(page)
    from lxml import html
    content = html.fromstring(page.content)
    links_found = 0
    for item in content.xpath("//div"):
        if "prp-indexpages-row" in item.classes:
            links_found += 1
            link = item.findall('.//a')[0].get('href')
            name = link.split('/')[::-1][0]
            links.append((name, "https://bn.wikisource.org/w/index.php?title=" + name + "&action=raw"))
    if links_found == 0:
        break
    i += 5000

<Response [200]>


<Response [200]>


In [18]:
links

[('Livre:Godefroy_-_Lexique.djvu',
  'https://fr.wikisource.org/w/index.php?title=Livre:Godefroy_-_Lexique.djvu&action=raw'),
 ('Livre:Jaur%C3%A8s_-_Histoire_socialiste,_IV.djvu',
  'https://fr.wikisource.org/w/index.php?title=Livre:Jaur%C3%A8s_-_Histoire_socialiste,_IV.djvu&action=raw'),
 ('Livre:Jaur%C3%A8s_-_Histoire_socialiste,_III.djvu',
  'https://fr.wikisource.org/w/index.php?title=Livre:Jaur%C3%A8s_-_Histoire_socialiste,_III.djvu&action=raw'),
 ('Livre:Dictionnaire_de_Tr%C3%A9voux,_1771,_I.djvu',
  'https://fr.wikisource.org/w/index.php?title=Livre:Dictionnaire_de_Tr%C3%A9voux,_1771,_I.djvu&action=raw'),
 ('Livre:Platon_-_%C5%92uvres,_trad._Cousin,_I_et_II.djvu',
  'https://fr.wikisource.org/w/index.php?title=Livre:Platon_-_%C5%92uvres,_trad._Cousin,_I_et_II.djvu&action=raw'),
 ('Livre:Jaur%C3%A8s_-_Histoire_socialiste,_I.djvu',
  'https://fr.wikisource.org/w/index.php?title=Livre:Jaur%C3%A8s_-_Histoire_socialiste,_I.djvu&action=raw'),
 ('Livre:Cervantes_-_L%E2%80%99Ing%C3%A9ni

In [2]:

from lxml import html
content = html.fromstring(page.content)
links = []
for item in content.xpath("//div"):
    if "prp-indexpages-row" in item.classes:
        link = item.findall('.//a')[0].get('href')
        name = link.split('/')[::-1][0]
        links.append((name, "https://fr.wikisource.org/w/index.php?title=" + name + "&action=raw"))

print(links[0])

IndexError: list index out of range

In [6]:
import re

page = requests.get(links[0][1])
print(page.content)
page.encoding = 'utf-8'
page_content = str(page.content).split('|', 1)[1]
headings = re.sub(r"(\|Remarks).*(\|Notes)", r'\1=\n\2', str(page.content)).split('|')
headings.remove(headings[0])
count = 0
s = ""
for heading in headings:
    prop, val = heading.split('=', 1)
    s += prop + "\t"
    print(prop)
print(s[:-1], file=open("bn-ws.tsv", "w+"))

b'{{:MediaWiki:Proofreadpage_index_template\n|Type=book\n|Title=\xe0\xa6\xaa\xe0\xa7\x8b\xe0\xa6\x95\xe0\xa6\xbe-\xe0\xa6\xae\xe0\xa6\xbe\xe0\xa6\x95\xe0\xa6\xa1\xe0\xa6\xbc\n|Language=bn\n|Volume=\n|Author=\xe0\xa6\x9c\xe0\xa6\x97\xe0\xa6\xa6\xe0\xa6\xbe\xe0\xa6\xa8\xe0\xa6\xa8\xe0\xa7\x8d\xe0\xa6\xa6 \xe0\xa6\xb0\xe0\xa6\xbe\xe0\xa6\xaf\xe0\xa6\xbc\n|Co-author1=\n|Co-author2=\n|Translator=\n|Co-translator1=\n|Co-translator2=\n|Editor=\n|Co-editor1=\n|Co-editor2=\n|Illustrator=\n|Publisher=\xe0\xa6\x87\xe0\xa6\xa3\xe0\xa7\x8d\xe0\xa6\xa1\xe0\xa6\xbf\xe0\xa6\xaf\xe0\xa6\xbc\xe0\xa6\xbe\xe0\xa6\xa8 \xe0\xa6\xaa\xe0\xa7\x8d\xe0\xa6\xb0\xe0\xa7\x87\xe0\xa6\xb8\n|Address=\xe0\xa6\x8f\xe0\xa6\xb2\xe0\xa6\xbe\xe0\xa6\xb9\xe0\xa6\xbe\xe0\xa6\xac\xe0\xa6\xbe\xe0\xa6\xa6\n|Year=1924\n|Key=\n|ISBN=\n|DLI=\n|WBPLN=\n|IA=\n|NLI=\n|Source=pdf\n|Image=5\n|Progress=T\n|Pages=<pagelist\n1to4=-\n5=\xe0\xa6\xaa\xe0\xa7\x8d\xe0\xa6\xb0\xe0\xa6\x9a\xe0\xa7\x8d\xe0\xa6\x9b\xe0\xa6\xa6\n6=\xe0\xa6\xaa\xe0\x

In [7]:
import codecs
from urllib.parse import unquote
import re
for y, l in links:
    page = requests.get(l)
    page.encoding = 'utf-8'
    page_content = str(page.content).split('|', 1)[1]
    headings = re.sub(r"(\|Remarks).*(\|Notes)", r'\1=\2', str(page.content))
    headings = re.sub(r"(\|Pages).*(\|Volumes)", r'\1=\2', headings)
    #print(headings)
    headings = headings.split('=')
    #print(headings)
    headings.remove(headings[0])
    s=unquote(y) + "\t" + "https://bn.wikisource.org/wiki/" + unquote(y) + "\t"
    for heading in headings:
        try:
            val, prop = heading.split('|', 1)
        except Exception:
            val = heading
        val = val.replace("}}","")
        val = val.replace("\\n", "")
        s += val + "\t"
    #print(s)
    with codecs.open("bn-ws.tsv", "a+", "utf-8-sig") as temp:
        temp.write(bytes(s, 'utf-8').decode('unicode_escape').encode('latin-1').decode('utf8') + "\n")

In [13]:
data = {'url': 'https://bn.wikisource.org/wiki/নির্ঘণ্ট:শ্রীক্ষেত্র-তত্ত্ব-সুধা.pdf'}
r = requests.post(url="https://meta.wikimedia.org/w/api.php?action=shortenurl", data=data)
print(r.text)

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>MediaWiki API result - Meta</title>
<script>document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"Special","wgCanonicalSpecialPageName":"ApiHelp","wgNamespaceNumber":-1,"wgPageName":"Special:ApiHelp","wgTitle":"ApiHelp","wgCurRevisionId":0,"wgRevisionId":0,"wgArticleId":0,"wgIsArticle":!1,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRelevantPag